<a id='1'></a>
# 1. Import packages

In [1]:
from keras.models import Model
from keras.layers import *
from keras.layers.advanced_activations import LeakyReLU
from keras.initializers import RandomNormal
from pixel_shuffler import PixelShuffler
import keras.backend as K
import tensorflow as tf

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
import mtcnn_detect_face
from umeyama import umeyama

<a id='2'></a>
# 2. Install requirements

## ========== CAUTION ========== 

If you are running this jupyter on local machine. Please read [this blog](http://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/) before running the following cells which pip install packages.

In [4]:
# https://github.com/ageitgey/face_recognition
#!pip install face_recognition

In [5]:
#!pip install moviepy

<a id='4'></a>
# 4. Config

In [5]:
K.set_learning_phase(0)

In [6]:
IMAGE_SHAPE = (64, 64, 3)
nc_in = 3 # number of input channels of generators
nc_D_inp = 6 # number of input channels of discriminators

use_self_attn = False
w_l2 = 1e-4 # weight decay

batchSize = 8

# Path of training images
img_dirA = './faceA/*.*'
img_dirB = './faceB/*.*'

<a id='5'></a>
# 5. Define models

In [7]:
conv_init = RandomNormal(0, 0.02)

In [8]:
class Scale(Layer):
    '''
    Code borrows from https://github.com/flyyufelix/cnn_finetune
    '''
    def __init__(self, weights=None, axis=-1, gamma_init='zero', **kwargs):
        self.axis = axis
        self.gamma_init = initializers.get(gamma_init)
        self.initial_weights = weights
        super(Scale, self).__init__(**kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(shape=input_shape)]

        # Compatibility with TensorFlow >= 1.0.0
        self.gamma = K.variable(self.gamma_init((1,)), name='{}_gamma'.format(self.name))
        self.trainable_weights = [self.gamma]

        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights

    def call(self, x, mask=None):
        return self.gamma * x

    def get_config(self):
        config = {"axis": self.axis}
        base_config = super(Scale, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


def self_attn_block(inp, nc):
    '''
    Code borrows from https://github.com/taki0112/Self-Attention-GAN-Tensorflow
    '''
    assert nc//8 > 0, f"Input channels must be >= 8, but got nc={nc}"
    x = inp
    shape_x = x.get_shape().as_list()
    
    f = Conv2D(nc//8, 1, kernel_initializer=conv_init)(x)
    g = Conv2D(nc//8, 1, kernel_initializer=conv_init)(x)
    h = Conv2D(nc, 1, kernel_initializer=conv_init)(x)
    
    shape_f = f.get_shape().as_list()
    shape_g = g.get_shape().as_list()
    shape_h = h.get_shape().as_list()
    flat_f = Reshape((-1, shape_f[-1]))(f)
    flat_g = Reshape((-1, shape_g[-1]))(g)
    flat_h = Reshape((-1, shape_h[-1]))(h)   
    
    s = Lambda(lambda x: tf.matmul(x[0], x[1], transpose_b=True))([flat_g, flat_f])

    beta = Softmax(axis=-1)(s)
    o = Lambda(lambda x: tf.matmul(x[0], x[1]))([beta, flat_h])
    o = Reshape(shape_x[1:])(o)
    o = Scale()(o)
    
    out = add([o, inp])
    return out

def conv_block(input_tensor, f):
    x = input_tensor
    x = Conv2D(f, kernel_size=3, strides=2, kernel_regularizer=regularizers.l2(w_l2),  
               kernel_initializer=conv_init, use_bias=False, padding="same")(x)
    x = Activation("relu")(x)
    return x

def conv_block_d(input_tensor, f, use_instance_norm=False):
    x = input_tensor
    x = Conv2D(f, kernel_size=4, strides=2, kernel_regularizer=regularizers.l2(w_l2), 
               kernel_initializer=conv_init, use_bias=False, padding="same")(x)
    x = LeakyReLU(alpha=0.2)(x)
    return x

def res_block(input_tensor, f):
    x = input_tensor
    x = Conv2D(f, kernel_size=3, kernel_regularizer=regularizers.l2(w_l2), 
               kernel_initializer=conv_init, use_bias=False, padding="same")(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(f, kernel_size=3, kernel_regularizer=regularizers.l2(w_l2), 
               kernel_initializer=conv_init, use_bias=False, padding="same")(x)
    x = add([x, input_tensor])
    x = LeakyReLU(alpha=0.2)(x)
    return x

def upscale_ps(filters, use_norm=True):
    def block(x):
        x = Conv2D(filters*4, kernel_size=3, kernel_regularizer=regularizers.l2(w_l2), 
                   kernel_initializer=RandomNormal(0, 0.02), padding='same')(x)
        x = LeakyReLU(0.2)(x)
        x = PixelShuffler()(x)
        return x
    return block

def Discriminator(nc_in, input_size=64):
    inp = Input(shape=(input_size, input_size, nc_in))
    #x = GaussianNoise(0.05)(inp)
    x = conv_block_d(inp, 64, False)
    x = conv_block_d(x, 128, False)
    x = self_attn_block(x, 128) if use_self_attn else x
    x = conv_block_d(x, 256, False)
    x = self_attn_block(x, 256) if use_self_attn else x
    out = Conv2D(1, kernel_size=4, kernel_initializer=conv_init, use_bias=False, padding="same")(x)   
    return Model(inputs=[inp], outputs=out)

def Encoder(nc_in=3, input_size=64):
    inp = Input(shape=(input_size, input_size, nc_in))
    x = Conv2D(64, kernel_size=5, kernel_initializer=conv_init, use_bias=False, padding="same")(inp)
    x = conv_block(x,128)
    x = conv_block(x,256)
    x = self_attn_block(x, 256) if use_self_attn else x
    x = conv_block(x,512) 
    x = self_attn_block(x, 512) if use_self_attn else x
    x = conv_block(x,1024)
    x = Dense(1024)(Flatten()(x))
    x = Dense(4*4*1024)(x)
    x = Reshape((4, 4, 1024))(x)
    out = upscale_ps(512)(x)
    return Model(inputs=inp, outputs=out)

def Decoder_ps(nc_in=512, input_size=8):
    input_ = Input(shape=(input_size, input_size, nc_in))
    x = input_
    x = upscale_ps(256)(x)
    x = upscale_ps(128)(x)
    x = self_attn_block(x, 128) if use_self_attn else x
    x = upscale_ps(64)(x)
    x = res_block(x, 64)
    x = self_attn_block(x, 64) if use_self_attn else x
    #x = Conv2D(4, kernel_size=5, padding='same')(x)   
    alpha = Conv2D(1, kernel_size=5, padding='same', activation="sigmoid")(x)
    rgb = Conv2D(3, kernel_size=5, padding='same', activation="tanh")(x)
    out = concatenate([alpha, rgb])
    return Model(input_, out)    

In [9]:
encoder = Encoder()
decoder_A = Decoder_ps()
decoder_B = Decoder_ps()

x = Input(shape=IMAGE_SHAPE)

netGA = Model(x, decoder_A(encoder(x)))
netGB = Model(x, decoder_B(encoder(x)))

<a id='6'></a>
# 6. Load Models

In [ ]:
try:
    encoder.load_weights("models/encoder.h5")
    decoder_A.load_weights("models/decoder_A.h5")
    decoder_B.load_weights("models/decoder_B.h5")
    print ("Model weights files are successfully loaded")
except:
    print ("!! Error occurs during loading weights files.")
    pass

<a id='7'></a>
# 7. Define Inputs/Outputs Variables

In [10]:
def cycle_variables(netG):
    distorted_input = netG.inputs[0]
    fake_output = netG.outputs[0]
    alpha = Lambda(lambda x: x[:,:,:, :1])(fake_output)
    rgb = Lambda(lambda x: x[:,:,:, 1:])(fake_output)
    
    masked_fake_output = alpha * rgb + (1-alpha) * distorted_input 

    fn_generate = K.function([distorted_input], [masked_fake_output])
    fn_mask = K.function([distorted_input], [concatenate([alpha, alpha, alpha])])
    fn_abgr = K.function([distorted_input], [concatenate([alpha, rgb])])
    return fn_generate, fn_mask, fn_abgr

In [11]:
path_A, path_mask_A, path_abgr_A = cycle_variables(netGA)
path_B, path_mask_B, path_abgr_B = cycle_variables(netGB)

<a id='12'></a>
# 12. Import modules for video making

Given a video as input, the following cells will detect face for each frame using dlib's cnn model. And use trained GAN model to transform detected face into target face. Then output a video with swapped faces.

In [14]:
# Download ffmpeg if need, which is required by moviepy.

#import imageio
#imageio.plugins.ffmpeg.download()

In [15]:
from moviepy.editor import VideoFileClip

# Define transform path: A2B or B2A

In [16]:
direction = "BtoA" # default trainsforming faceB to faceA

if direction is "AtoB":
    path_func = path_abgr_B
elif direction is "BtoA":
    path_func = path_abgr_A
else:
    print ("direction should be either AtoB or BtoA")

# MTCNN setup

In [17]:
def create_mtcnn(sess, model_path):
    if not model_path:
        model_path,_ = os.path.split(os.path.realpath(__file__))

    with tf.variable_scope('pnet2'):
        data = tf.placeholder(tf.float32, (None,None,None,3), 'input')
        pnet = mtcnn_detect_face.PNet({'data':data})
        pnet.load(os.path.join(model_path, 'det1.npy'), sess)
    with tf.variable_scope('rnet2'):
        data = tf.placeholder(tf.float32, (None,24,24,3), 'input')
        rnet = mtcnn_detect_face.RNet({'data':data})
        rnet.load(os.path.join(model_path, 'det2.npy'), sess)
    with tf.variable_scope('onet2'):
        data = tf.placeholder(tf.float32, (None,48,48,3), 'input')
        onet = mtcnn_detect_face.ONet({'data':data})
        onet.load(os.path.join(model_path, 'det3.npy'), sess)
    return pnet, rnet, onet

In [18]:
WEIGHTS_PATH = "./mtcnn_weights/"

sess = K.get_session()
with sess.as_default():
    global pnet, rnet, onet 
    pnet2, rnet2, onet2 = create_mtcnn(sess, WEIGHTS_PATH)

In [19]:
global pnet, rnet, onet

In [20]:
pnet_fun = K.function([pnet2.layers['data']],[pnet2.layers['conv4-2'], pnet2.layers['prob1']])
rnet_fun = K.function([rnet2.layers['data']],[rnet2.layers['conv5-2'], rnet2.layers['prob1']])
onet_fun = K.function([onet2.layers['data']], [onet2.layers['conv6-2'], onet2.layers['conv6-3'], onet2.layers['prob1']])

In [21]:
with tf.variable_scope('pnet2', reuse=True):
    data = tf.placeholder(tf.float32, (None,None,None,3), 'input')
    pnet2 = mtcnn_detect_face.PNet({'data':data})
    pnet2.load(os.path.join("./mtcnn_weights/", 'det1.npy'), sess)
with tf.variable_scope('rnet2', reuse=True):
    data = tf.placeholder(tf.float32, (None,24,24,3), 'input')
    rnet2 = mtcnn_detect_face.RNet({'data':data})
    rnet2.load(os.path.join("./mtcnn_weights/", 'det2.npy'), sess)
with tf.variable_scope('onet2', reuse=True):
    data = tf.placeholder(tf.float32, (None,48,48,3), 'input')
    onet2 = mtcnn_detect_face.ONet({'data':data})
    onet2.load(os.path.join("./mtcnn_weights/", 'det3.npy'), sess)

In [22]:
pnet = K.function([pnet2.layers['data']],[pnet2.layers['conv4-2'], pnet2.layers['prob1']])
rnet = K.function([rnet2.layers['data']],[rnet2.layers['conv5-2'], rnet2.layers['prob1']])
onet = K.function([onet2.layers['data']], [onet2.layers['conv6-2'], onet2.layers['conv6-3'], onet2.layers['prob1']])

<a id='13'></a>
# 13. Make video clips

### Default transform: face B to face A

In [23]:
use_smoothed_mask = True
use_smoothed_bbox = True

def is_overlap(box1, box2):
    overlap_x0 = np.max([box1[0], box2[0]]).astype(np.float32)
    overlap_y1 = np.min([box1[1], box2[1]]).astype(np.float32)
    overlap_x1 = np.min([box1[2], box2[2]]).astype(np.float32)
    overlap_y0 = np.max([box1[3], box2[3]]).astype(np.float32)
    area_iou = (overlap_x1-overlap_x0) * (overlap_y1-overlap_y0)
    area_box1 = (box1[2]-box1[0]) * (box1[1]-box1[3])
    area_box2 = (box2[2]-box2[0]) * (box2[1]-box2[3])    
    return (area_iou / area_box1) >= 0.2
    
def remove_overlaps(faces):    
    main_face = get_most_conf_face(faces)
    main_face_bbox = main_face[0]
    result_faces = []
    result_faces.append(main_face_bbox)
    for (x0, y1, x1, y0, conf_score) in faces:
        if not is_overlap(main_face_bbox, (x0, y1, x1, y0)):
            result_faces.append((x0, y1, x1, y0, conf_score))
    return result_faces

def get_most_conf_face(faces):
    # Return the bbox w/ the highest confidence score
    best_conf_score = 0
    conf_face = None
    for (x0, y1, x1, y0, conf_score) in faces: 
        if conf_score >= best_conf_score:
            best_conf_score = conf_score
            conf_face = [(x0, y1, x1, y0, conf_score)]
    return conf_face

def kalmanfilter_init(noise_coef):
    kf = cv2.KalmanFilter(4,2)
    kf.measurementMatrix = np.array([[1,0,0,0],[0,1,0,0]], np.float32)
    kf.transitionMatrix = np.array([[1,0,1,0],[0,1,0,1],[0,0,1,0],[0,0,0,1]], np.float32)
    kf.processNoiseCov = noise_coef * np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]], np.float32)
    return kf

def is_higher_than_480p(x):
    return (x.shape[0] * x.shape[1]) >= (858*480)

def is_higher_than_720p(x):
    return (x.shape[0] * x.shape[1]) >= (1280*720)

def is_higher_than_1080p(x):
    return (x.shape[0] * x.shape[1]) >= (1920*1080)

def calibrate_coord(faces, video_scaling_factor):
    for i, (x0, y1, x1, y0, _) in enumerate(faces):
        faces[i] = (x0*video_scaling_factor, y1*video_scaling_factor, 
                    x1*video_scaling_factor, y0*video_scaling_factor, _)
    return faces

def process_mtcnn_bbox(bboxes, im_shape):
    # outuut bbox coordinate of MTCNN is (y0, x0, y1, x1)
    # Process the bbox coord. to a square bbox with ordering (x0, y1, x1, y0)
    for i, bbox in enumerate(bboxes):
        y0, x0, y1, x1 = bboxes[i,0:4]
        w = int(y1 - y0)
        h = int(x1 - x0)
        length = (w + h)/2
        center = (int((x1+x0)/2),int((y1+y0)/2))
        new_x0 = np.max([0, (center[0]-length//2)])#.astype(np.int32)
        new_x1 = np.min([im_shape[0], (center[0]+length//2)])#.astype(np.int32)
        new_y0 = np.max([0, (center[1]-length//2)])#.astype(np.int32)
        new_y1 = np.min([im_shape[1], (center[1]+length//2)])#.astype(np.int32)
        bboxes[i,0:4] = new_x0, new_y1, new_x1, new_y0
    return bboxes

def get_faces_bbox(image):  
    global pnet, rnet, onet 
    global detec_threshold
    minsize = 20 # minimum size of face
    threshold = [ 0.6, 0.7, detec_threshold ]  # three steps's threshold
    factor = 0.709 # scale factor
    if manually_downscale:
        video_scaling_factor = manually_downscale_factor
        resized_image = cv2.resize(image, 
                                   (image.shape[1]//video_scaling_factor, 
                                    image.shape[0]//video_scaling_factor))
        faces, pnts = mtcnn_detect_face.detect_face(resized_image, minsize, pnet, rnet, onet, threshold, factor)
        faces = process_mtcnn_bbox(faces, resized_image.shape)
        faces = calibrate_coord(faces, video_scaling_factor)
    elif is_higher_than_1080p(image):
        video_scaling_factor = 4 + video_scaling_offset
        resized_image = cv2.resize(image, 
                                   (image.shape[1]//video_scaling_factor, 
                                    image.shape[0]//video_scaling_factor))
        faces, pnts = mtcnn_detect_face.detect_face(resized_image, minsize, pnet, rnet, onet, threshold, factor)
        faces = process_mtcnn_bbox(faces, resized_image.shape)
        faces = calibrate_coord(faces, video_scaling_factor)
    elif is_higher_than_720p(image):
        video_scaling_factor = 3 + video_scaling_offset
        resized_image = cv2.resize(image, 
                                   (image.shape[1]//video_scaling_factor, 
                                    image.shape[0]//video_scaling_factor))
        faces, pnts = mtcnn_detect_face.detect_face(resized_image, minsize, pnet, rnet, onet, threshold, factor)
        faces = process_mtcnn_bbox(faces, resized_image.shape)
        faces = calibrate_coord(faces, video_scaling_factor)  
    elif is_higher_than_480p(image):
        video_scaling_factor = 2 + video_scaling_offset
        resized_image = cv2.resize(image, 
                                   (image.shape[1]//video_scaling_factor, 
                                    image.shape[0]//video_scaling_factor))
        faces, pnts = mtcnn_detect_face.detect_face(resized_image, minsize, pnet, rnet, onet, threshold, factor)
        faces = process_mtcnn_bbox(faces, resized_image.shape)
        faces = calibrate_coord(faces, video_scaling_factor)
    else:
        faces, pnts = mtcnn_detect_face.detect_face(image, minsize, pnet, rnet, onet, threshold, factor)
        faces = process_mtcnn_bbox(faces, image.shape)
    return faces

def get_smoothed_coord(x0, x1, y0, y1, shape, ratio=0.65):
    global prev_x0, prev_x1, prev_y0, prev_y1
    if not use_kalman_filter:
        x0 = int(ratio * prev_x0 + (1-ratio) * x0)
        x1 = int(ratio * prev_x1 + (1-ratio) * x1)
        y1 = int(ratio * prev_y1 + (1-ratio) * y1)
        y0 = int(ratio * prev_y0 + (1-ratio) * y0)
    else:
        x0y0 = np.array([x0, y0]).astype(np.float32)
        x1y1 = np.array([x1, y1]).astype(np.float32)
        kf0.correct(x0y0)
        pred_x0y0 = kf0.predict()
        kf1.correct(x1y1)
        pred_x1y1 = kf1.predict()
        x0 = np.max([0, pred_x0y0[0][0]]).astype(np.int)
        x1 = np.min([shape[0], pred_x1y1[0][0]]).astype(np.int)
        y0 = np.max([0, pred_x0y0[1][0]]).astype(np.int)
        y1 = np.min([shape[1], pred_x1y1[1][0]]).astype(np.int)
        if x0 == x1 or y0 == y1:
            x0, y0, x1, y1 = prev_x0, prev_y0, prev_x1, prev_y1
    return x0, x1, y0, y1     
    
def set_global_coord(x0, x1, y0, y1):
    global prev_x0, prev_x1, prev_y0, prev_y1
    prev_x0 = x0
    prev_x1 = x1
    prev_y1 = y1
    prev_y0 = y0
    
def generate_face(ae_input, path_abgr, roi_size, roi_image):
    result = np.squeeze(np.array([path_abgr([[ae_input]])]))
    result_a = result[:,:,0] * 255
    result_bgr = np.clip( (result[:,:,1:] + 1) * 255 / 2, 0, 255 )
    result_a_clear = np.copy(result_a)
    result_a = cv2.GaussianBlur(result_a ,(7,7),6)
    if use_landmark_match and False:
        resized_roi = cv2.resize(roi_image, (64,64))
        result_bgr, result_a = landmarks_match_mtcnn(resized_roi, result_bgr, result_a)
    result_a = np.expand_dims(result_a, axis=2)
    result = (result_a/255 * result_bgr + (1 - result_a/255) * ((ae_input + 1) * 255 / 2)).astype('uint8')
    if use_color_correction:
        result = color_hist_match(result, roi_image)
    result = cv2.cvtColor(result.astype('uint8'), cv2.COLOR_BGR2RGB)
    result = cv2.resize(result, (roi_size[1],roi_size[0]))
    result_a_clear = np.expand_dims(cv2.resize(result_a_clear, (roi_size[1],roi_size[0])), axis=2)
    return result, result_a_clear

def get_init_mask_map(image):
    return np.zeros_like(image)

def get_init_comb_img(input_img):
    comb_img = np.zeros([input_img.shape[0], input_img.shape[1]*2,input_img.shape[2]])
    comb_img[:, :input_img.shape[1], :] = input_img
    comb_img[:, input_img.shape[1]:, :] = input_img
    return comb_img    

def get_init_triple_img(input_img, no_face=False):
    if no_face:
        triple_img = np.zeros([input_img.shape[0], input_img.shape[1]*3,input_img.shape[2]])
        triple_img[:, :input_img.shape[1], :] = input_img
        triple_img[:, input_img.shape[1]:input_img.shape[1]*2, :] = input_img      
        triple_img[:, input_img.shape[1]*2:, :] = (input_img * .15).astype('uint8')  
        return triple_img
    else:
        triple_img = np.zeros([input_img.shape[0], input_img.shape[1]*3,input_img.shape[2]])
        return triple_img

def get_mask(roi_image, h, w):
    mask = np.zeros_like(roi_image)
    mask[h//15:-h//15,w//15:-w//15,:] = 255
    mask = cv2.GaussianBlur(mask,(15,15),10)
    return mask

def hist_match(source, template):
    # Code borrow from:
    # https://stackoverflow.com/questions/32655686/histogram-matching-of-two-images-in-python-2-x
    oldshape = source.shape
    source = source.ravel()
    template = template.ravel()
    s_values, bin_idx, s_counts = np.unique(source, return_inverse=True,
                                            return_counts=True)
    t_values, t_counts = np.unique(template, return_counts=True)

    s_quantiles = np.cumsum(s_counts).astype(np.float64)
    s_quantiles /= s_quantiles[-1]
    t_quantiles = np.cumsum(t_counts).astype(np.float64)
    t_quantiles /= t_quantiles[-1]
    interp_t_values = np.interp(s_quantiles, t_quantiles, t_values)

    return interp_t_values[bin_idx].reshape(oldshape)

def color_hist_match(src_im, tar_im):
    #src_im = cv2.cvtColor(src_im, cv2.COLOR_BGR2Lab)
    #tar_im = cv2.cvtColor(tar_im, cv2.COLOR_BGR2Lab)
    matched_R = hist_match(src_im[:,:,0], tar_im[:,:,0])
    matched_G = hist_match(src_im[:,:,1], tar_im[:,:,1])
    matched_B = hist_match(src_im[:,:,2], tar_im[:,:,2])
    matched = np.stack((matched_R, matched_G, matched_B), axis=2).astype(np.float64)
    return matched

def landmarks_match_mtcnn(source, target, alpha):
    global prev_pnts1, prev_pnts2
    ratio = 0.2
    """
    TODO: Reuse the landmarks of source image. Conceivable bug: coordinate mismatch.
    """
    minsize = 20 # minimum size of face
    threshold = [ 0.6, 0.7, 0.93 ]  # three steps's threshold
    factor = 0.709 # scale factor
    _, pnts1 = mtcnn_detect_face.detect_face(source, minsize, pnet, rnet, onet, threshold, factor) # redundant detection
    _, pnts2 = mtcnn_detect_face.detect_face(target, minsize, pnet, rnet, onet, threshold, factor)  
    
    if len(prev_pnts1) == 0 and len(prev_pnts2) == 0:
        if pnts1.shape[0] == 10 and pnts2.shape[0] == 10:
            prev_pnts1, prev_pnts2 = pnts1, pnts2        
    try:
        landmarks_XY1 = []
        landmarks_XY2 = []
        for i in range(5):
            landmarks_XY1.extend([((1-ratio)*pnts1[i+5][0] + ratio*prev_pnts1[i+5][0], 
                                   (1-ratio)*pnts1[i][0] + ratio*prev_pnts1[i][0])])
            landmarks_XY2.extend([((1-ratio)*pnts2[i+5][0] + ratio*prev_pnts2[i+5][0], 
                                   (1-ratio)*pnts2[i][0] + ratio*prev_pnts2[i][0])])
        M = umeyama(np.array(landmarks_XY1), np.array(landmarks_XY2), True)[0:2]
        result = cv2.warpAffine(source, M, (64, 64), borderMode=cv2.BORDER_REPLICATE)  
        mask = np.stack([alpha, alpha, alpha], axis=2)
        assert len(mask.shape) == 3, "len(mask.shape) is " + str(len(mask.shape))
        mask = cv2.warpAffine(mask, M, (64, 64), borderMode=cv2.BORDER_REPLICATE) 
        prev_landmarks_XY1, prev_landmarks_XY2 = landmarks_XY1, landmarks_XY2
        return result, mask[:,:,0].astype(np.float32)
    except:
        return source, alpha

def process_video(input_img): 
    global prev_x0, prev_x1, prev_y0, prev_y1
    global frames      
    global pnet, rnet, onet
    """
    The following if statement is meant to solve a bug that has an unknow cause.
    """
    if frames <= 2:
        with tf.variable_scope('pnet2', reuse=True):
            pnet2 = None
            data = tf.placeholder(tf.float32, (None,None,None,3), 'input')
            pnet2 = mtcnn_detect_face.PNet({'data':data})
            pnet2.load(os.path.join("./mtcnn_weights/", 'det1.npy'), sess)
        with tf.variable_scope('rnet2', reuse=True):
            rnet2 = None
            data = tf.placeholder(tf.float32, (None,24,24,3), 'input')
            rnet2 = mtcnn_detect_face.RNet({'data':data})
            rnet2.load(os.path.join("./mtcnn_weights/", 'det2.npy'), sess)
        with tf.variable_scope('onet2', reuse=True):
            onet2 = None
            data = tf.placeholder(tf.float32, (None,48,48,3), 'input')
            onet2 = mtcnn_detect_face.ONet({'data':data})
            onet2.load(os.path.join("./mtcnn_weights/", 'det3.npy'), sess)
        pnet = K.function([pnet2.layers['data']],
                          [pnet2.layers['conv4-2'], 
                           pnet2.layers['prob1']])
        rnet = K.function([rnet2.layers['data']],
                          [rnet2.layers['conv5-2'], 
                           rnet2.layers['prob1']])
        onet = K.function([onet2.layers['data']], 
                          [onet2.layers['conv6-2'], 
                           onet2.layers['conv6-3'], 
                           onet2.layers['prob1']])
    
    image = input_img
    faces = get_faces_bbox(image) # faces: face bbox coord
    
    if len(faces) == 0:
        comb_img = get_init_comb_img(input_img)
        triple_img = get_init_triple_img(input_img, no_face=True)
    else:
        faces = remove_overlaps(faces) # Has non-max suppress already been implemented in MTCNN?
        
    mask_map = get_init_mask_map(image)
    comb_img = get_init_comb_img(input_img)
    best_conf_score = 0
    
    for (x0, y1, x1, y0, conf_score) in faces:    
        #print (x0, y1, x1, y0, conf_score)
        # smoothing bounding box
        if use_smoothed_bbox:
            if frames != 0 and conf_score >= best_conf_score:
                x0, x1, y0, y1 = get_smoothed_coord(x0, x1, y0, y1, 
                                                    image.shape, 
                                                    ratio=0.65 if use_kalman_filter else bbox_moving_avg_coef)
                set_global_coord(x0, x1, y0, y1)
                best_conf_score = conf_score
                frames += 1
            elif conf_score <= best_conf_score:
                frames += 1
            else:
                if conf_score >= best_conf_score:
                    set_global_coord(x0, x1, y0, y1)
                    best_conf_score = conf_score
                if use_kalman_filter:
                    for i in range(200):
                        kf0.predict()
                        kf1.predict()
                frames += 1
                
        h = int(x1 - x0)
        w = int(y1 - y0)
        roi_coef = 15
        roi_x0, roi_x1, roi_y0, roi_y1 = int(x0+h//roi_coef), int(x1-h//roi_coef), int(y0+w//roi_coef), int(y1-w//roi_coef)
            
        cv2_img = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        roi_image = cv2_img[roi_x0:roi_x1,roi_y0:roi_y1,:]
        roi_size = roi_image.shape  
        
        ae_input = cv2.resize(roi_image, (64,64))/255. * 2 - 1  
        np.squeeze(np.array([path_abgr_A([[ae_input]])]))
        result, result_a = generate_face(ae_input, path_func, roi_size, roi_image)
        if conf_score >= best_conf_score:
            mask_map[roi_x0:roi_x1,roi_y0:roi_y1,:] = result_a
            mask_map = np.clip(mask_map + .15 * input_img, 0, 255)     
        else:
            mask_map[roi_x0:roi_x1,roi_y0:roi_y1,:] += result_a
            mask_map = np.clip(mask_map, 0, 255)
        
        if use_smoothed_mask:
            mask = get_mask(roi_image, h, w)
            roi_rgb = cv2.cvtColor(roi_image, cv2.COLOR_BGR2RGB)
            smoothed_result = mask/255 * result + (1-mask/255) * roi_rgb
            comb_img[roi_x0:roi_x1, input_img.shape[1]+roi_y0:input_img.shape[1]+roi_y1,:] = smoothed_result
        else:
            comb_img[roi_x0:roi_x1, input_img.shape[1]+roi_y0:input_img.shape[1]+roi_y1,:] = result
            
        triple_img = get_init_triple_img(input_img)
        triple_img[:, :input_img.shape[1]*2, :] = comb_img
        triple_img[:, input_img.shape[1]*2:, :] = mask_map
        
    global output_type
    if output_type == 1:
        return comb_img[:, input_img.shape[1]:, :]  # return only result image
    elif output_type == 2:
        return comb_img  # return input and result image combined as one
    elif output_type == 3:
        return triple_img #return input,result and mask heatmap image combined as one

### Video making config

**Description**
```python
    video_scaling_offset = 0 # Increase by 1 if OOM happens.
    manually_downscale = False # Set True if increasing offset doesn't help
    manually_downscale_factor = int(2) # Increase by 1 if OOM still happens.
    use_color_correction = False # Option for color corretion
```

In [24]:
use_kalman_filter = False
if use_kalman_filter:
    noise_coef = 5e-3 # Increase by 10x if tracking is slow. 
    kf0 = kalmanfilter_init(noise_coef)
    kf1 = kalmanfilter_init(noise_coef)
else:
    bbox_moving_avg_coef = 0.65
    
video_scaling_offset = 0 
manually_downscale = False
manually_downscale_factor = int(2) # should be an positive integer
use_color_correction = False
use_landmark_match = False # Under developement, This is not functioning.

# ========== Change the following line for different output type==========
# Output type: 
#    1. [ result ] 
#    2. [ source | result ] 
#    3. [ source | result | mask ]
global output_type
output_type = 3

# Detection threshold:  a float point between 0 and 1. Decrease this value if faces are missed.
global detec_threshold
detec_threshold = 0.7

### Generate video clip

In [ ]:
# Variables for smoothing bounding box
global prev_x0, prev_x1, prev_y0, prev_y1
global frames
global prev_pnts1, prev_pnts2
prev_x0 = prev_x1 = prev_y0 = prev_y1 = 0
frames = 0
prev_pnts1 = prev_pnts2 = np.array([])

output = 'OUTPUT_VIDEO.mp4'
clip1 = VideoFileClip("INPUT_VIDEO.mp4")
clip = clip1.fl_image(process_video)#.subclip(7.5, 9) #NOTE: this function expects color images!!
%time clip.write_videofile(output, audio=False)

[MoviePy] >>>> Building video OUTPUT_VIDEO.mp4
[MoviePy] Writing video OUTPUT_VIDEO.mp4


  1%|          | 1/91 [00:06<10:09,  6.78s/it]

In [73]:
import gc
gc.collect()

0